In [26]:
import polars as pl
import pandas as pd
import fastexcel
import os
from ddf_utils.str import to_concept_id, format_float_digits

In [6]:
from functools import partial

In [27]:
sheet_loader = partial(pl.read_excel, skip_rows=2)

In [28]:
number_formatter = partial(format_float_digits, digits=8)

In [29]:
source_file = '../source/Statistical Review of World Energy Data.xlsx'

In [30]:
reader = fastexcel.read_excel(source_file)

sheets = reader.sheet_names

In [18]:
sheets

['Contents',
 'Total Energy Supply (TES) -EJ',
 'TES by fuel',
 'TES per Capita',
 'CO2 from Energy',
 'Natural Gas Flaring',
 'CO2 from Flaring',
 'CO2-Process Emissions, Methane',
 'CO2e Emissions ',
 'CCUS Capture Capacity',
 'Carbon Prices',
 'Oil - Proved reserves',
 'Oil - Proved reserves history',
 'Oil Production - barrels',
 'Oil Production - tonnes',
 'Crude+cond production - barrels',
 'NGLs production - barrels',
 'Liquids Consumption - barrels',
 'Oil Consumption - barrels',
 'Oil Consumption - Tonnes',
 'Oil Consumption - EJ',
 'Oil - Regional Consumption',
 'Spot crude prices',
 'Oil crude prices since 1861',
 'Oil refinery - throughput',
 'Oil refinery - capacity',
 'Oil - Regional refining margins',
 'Oil trade movements',
 'Oil - Inter-area movements',
 'Oil - Trade movements in 23-24',
 'Gas - Proved reserves',
 'Gas - Proved reserves history ',
 'Gas Production - Bcm',
 'Gas Production - bcf',
 'Gas Production - EJ',
 'Gas Consumption - Bcm',
 'Gas Consumption - Bcf

In [60]:
# drop a few non data tables
sheets.remove("Contents")
sheets.remove("Approximate conversion factors")
sheets.remove("Definitions")

In [67]:
# read the indicator name
def read_indicator_name(sheet_name):
    val = reader.load_sheet_by_name(sheet_name, n_rows=1, header_row=None, use_columns="A").to_polars().item()
    return val.replace("*", "")

In [69]:
read_indicator_name(sheets[1])

'Total energy supply: by fuel'

In [70]:
indicators = []

for s in sheets:
    indicators.append({"indicator_name": read_indicator_name(s), "sheet_name": s})

In [71]:
indicators

[{'indicator_name': 'Total energy supply',
  'sheet_name': 'Total Energy Supply (TES) -EJ'},
 {'indicator_name': 'Total energy supply: by fuel',
  'sheet_name': 'TES by fuel'},
 {'indicator_name': 'Total energy supply per capita',
  'sheet_name': 'TES per Capita'},
 {'indicator_name': 'Carbon Dioxide Emissions from Energy',
  'sheet_name': 'CO2 from Energy'},
 {'indicator_name': 'Natural Gas: Flaring',
  'sheet_name': 'Natural Gas Flaring'},
 {'indicator_name': 'Carbon Dioxide Emissions from Flaring',
  'sheet_name': 'CO2 from Flaring'},
 {'indicator_name': 'Carbon Dioxide Equivalent Emissions from Industrial Processes and Methane',
  'sheet_name': 'CO2-Process Emissions, Methane'},
 {'indicator_name': 'Carbon Dioxide Equivalent Emissions from Energy, Process Emissions, Methane, and Flaring ',
  'sheet_name': 'CO2e Emissions '},
 {'indicator_name': 'Carbon Capture, Usage, and Storage (CCUS)',
  'sheet_name': 'CCUS Capture Capacity'},
 {'indicator_name': 'Carbon Prices', 'sheet_name': '

there are a few formats in the excel

```
1. indicators with only geo, time dimension. Example:

Primary Energy: Consumption - Mtoe (from 1965),Primary Energy Consumption

2. geo/time/fuel. Example: 

PE Cons by fuel (old meth)

3. history prices. Example:

Oil: Spot crude prices ,Oil - Spot crude prices

4. refining margins:

Oil: Regional refining margins (from 1992),Oil - Regional refining margins

5. trade movments:

Oil: Trade movements (from 1980),Oil - Trade movements

```


we will focus on the first one, and create indicators by geo time.

In [77]:
df = reader.load_sheet_by_name("Wind Consumption - EJ", header_row=2).to_pandas()

In [49]:
df

,Exajoules,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2018,2019,2020,2021,2022,2023,2024,2024_1,2014-24,2024_2
0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
1,Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.120497,0.118352,0.128752,0.134268,0.142120,0.145838,0.168587,0.152835322,0.13834366,0.018650
2,Mexico,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.044767,0.060217,0.070930,0.075870,0.073903,0.074520,0.071118,-0.04826069,0.118851005,0.007867
3,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.991518,1.075936,1.228866,1.375261,1.579263,1.531423,1.648924,0.073784897,0.095793034,0.182409
4,Total North America,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.156782,1.254505,1.428548,1.585399,1.795286,1.751781,1.888630,0.075174165,0.099744722,0.208926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,♦ Less than 0.05%.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
117,n/a not available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
118,"USSR includes CIS, Georgia, Ukraine and the Ba...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
119,"# Excludes Estonia, Latvia and Lithuania pri...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN


In [74]:
last_col = "2024"  # change this according to the file.

# FIXME: 
# 1. sometimes there will be multiple "total world" in one sheet. 
# it indicates multiple break down of the same data available. 
# we need to consider how to handle such cases.
# I think they are all about biofeul indicators

# 2. some of the sheets' header row is not 2. we should have some way to configure it.
def read_sheet_and_preprocess(sheet_name):
    """preprocessing the data:
    1. rename the first column to geo_name
    2. rename the geo_name to alphanumeric
    3. drop all empty lines and lines after 'total world'

    Note: This function only applies to the tab with country as row index
    and year as column index.
    """
    data = reader.load_sheet_by_name(sheet_name, header_row=2).to_pandas()
    data = data.rename(columns={data.columns[0]: 'geo_name'})
    data['geo'] = data['geo_name'].map(to_concept_id)
    data = data.set_index('geo')
    data = data.dropna(how='all')
    try:
        data = data.loc[:'total_world', :last_col]
        data = data.reset_index()
        return data
    except KeyError as e:
        print(f"error: {e}")
        return None

In [224]:
# def apply_map(x, m):
#     """general function to replace value in a Series."""
#     if x in m:
#         return m[x]
#     return x

In [45]:
read_sheet_and_preprocess("Wind Consumption - EJ")

,geo,geo_name,1965,1966,1967,1968,1969,1970,1971,1972,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,canada,Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.097079,0.111359,0.113449,0.120497,0.118352,0.128752,0.134268,0.142120,0.145838,0.168587
1,mexico,Mexico,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.031482,0.037361,0.038231,0.044767,0.060217,0.070930,0.075870,0.073903,0.074520,0.071118
2,us,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.693522,0.825427,0.924737,0.991518,1.075936,1.228866,1.375261,1.579263,1.531423,1.648924
3,total_north_america,Total North America,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.822083,0.974148,1.076417,1.156782,1.254505,1.428548,1.585399,1.795286,1.751781,1.888630
4,argentina,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002136,0.001970,0.002202,0.005086,0.017985,0.033878,0.046575,0.050994,0.052123,0.058203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,thailand,Thailand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001181,0.001379,0.003992,0.005908,0.013212,0.011592,0.012715,0.010757,0.013468,0.012183
96,vietnam,Vietnam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000436,0.000724,0.001260,0.001753,0.002606,0.003535,0.012028,0.032728,0.041710,0.045889
97,other_asia_pacific,Other Asia Pacific,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000842,0.000788,0.000773,0.001441,0.001927,0.001783,0.002141,0.001936,0.002062,0.002158
98,total_asia_pacific,Total Asia Pacific,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.874811,1.120418,1.395170,1.666865,1.844371,2.069981,2.813797,3.245059,3.764021,4.188181


In [225]:
# create datapoint format dataframe.
def create_datapoints(sheet_name):
    data = read_sheet_and_preprocess(sheet_name)
    data = data.dropna(axis=1, how='all')
    data = data.drop('geo_name', axis=1)
    
    data = data.set_index('geo')

    d = data.T.unstack()
    d = d.dropna()
    d = d.reset_index()
    d.columns = ['geo', 'year', ddf_id]
    d[ddf_id] = d[ddf_id].map(number_formatter)

    return d.sort_values(by=['geo', 'year'])

In [72]:
read_sheet_and_preprocess(indicators[0]["sheet_name"])

,geo,geo_name,1965,1966,1967,1968,1969,1970,1971,1972,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,canada,Canada,4.168670,4.400851,4.623239,4.965211,5.222350,5.613447,5.796121,6.192707,...,12.122438,11.931747,12.093814,12.362536,12.326985,11.482506,11.596373,11.868844,11.805624,11.868849
1,mexico,Mexico,0.993530,1.046816,1.061961,1.152088,1.267891,1.348802,1.426723,1.581068,...,7.882033,8.129527,8.312085,8.107065,8.003264,7.237533,7.615490,7.968530,8.026493,8.044830
2,us,US,50.581347,53.483057,55.208237,58.594754,61.514629,63.776412,65.035982,68.356730,...,91.558942,91.151838,91.074934,94.411122,93.572743,86.213756,90.785549,92.251548,91.197763,91.832112
3,total_north_america,Total North America,55.743546,58.930725,60.893438,64.712052,68.004870,70.738661,72.258825,76.130505,...,111.563413,111.213112,111.480833,114.880723,113.902991,104.933795,109.997412,112.088922,111.029880,111.745791
4,argentina,Argentina,1.174068,1.212287,1.247564,1.288489,1.333015,1.261548,1.338143,1.355382,...,3.445617,3.450656,3.406040,3.360881,3.150608,2.925857,3.228706,3.386580,3.348011,3.273451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,thailand,Thailand,0.104619,0.124241,0.140404,0.182600,0.195939,0.229595,0.253833,0.313362,...,4.914411,5.009971,5.090490,5.233778,5.230736,4.884704,4.908795,4.903826,4.900233,5.019555
96,vietnam,Vietnam,0.103643,0.182587,0.252358,0.261456,0.309000,0.322089,0.286643,0.279031,...,2.574856,2.768675,2.903853,3.388560,3.927185,3.927862,3.774475,3.899876,4.274758,4.585856
97,other_asia_pacific,Other Asia Pacific,0.652919,0.694077,0.754460,0.813952,0.865515,0.926354,1.019342,1.077169,...,2.071199,2.158692,2.693644,2.852686,3.059364,3.103229,2.980404,3.123378,3.238434,3.290135
98,total_asia_pacific,Total Asia Pacific,17.841389,19.620125,20.502033,21.977654,25.328270,29.133652,31.657387,33.897443,...,219.345414,221.875222,229.404990,238.369554,245.561504,243.270288,255.487657,260.693980,271.434396,279.239316


In [75]:
indicator_data = []

for v in indicators:
    print(v["sheet_name"])
    indicator_data = read_sheet_and_preprocess(v["sheet_name"])

Total Energy Supply (TES) -EJ
TES by fuel
error: '2024'
TES per Capita
CO2 from Energy
Natural Gas Flaring
CO2 from Flaring
CO2-Process Emissions, Methane
CO2e Emissions 
CCUS Capture Capacity
Carbon Prices
error: '2024'
Oil - Proved reserves
error: '2024'
Oil - Proved reserves history
error: '2024'
Oil Production - barrels
Oil Production - tonnes
Crude+cond production - barrels
NGLs production - barrels
Liquids Consumption - barrels
Oil Consumption - barrels
Oil Consumption - Tonnes
Oil Consumption - EJ
Oil - Regional Consumption
Spot crude prices
error: '2024'
Oil crude prices since 1861
error: '2024'
Oil refinery - throughput
Oil refinery - capacity


Could not determine dtype for column 6, falling back to string


Oil - Regional refining margins
error: '2024'
Oil trade movements
error: "Cannot get right slice bound for non-unique label: 'total_world'"
Oil - Inter-area movements
error: '2024'
Oil - Trade movements in 23-24
error: '2024'
Gas - Proved reserves
error: '2024'
Gas - Proved reserves history 
error: '2024'
Gas Production - Bcm
Gas Production - bcf
Gas Production - EJ
Gas Consumption - Bcm
Gas Consumption - Bcf
Gas Consumption - EJ
Gas - H2 Production Capacity
Gas Prices 
error: '2024'
Gas - Trade movements
error: 'total_world'
Gas - LNG imports bcm
Gas - LNG exports bcm
error: 'total_world'
Gas trade 2024 - LNG
error: '2024'
Gas trade 2024 - pipeline
error: '2024'
Coal - Reserves
error: '2024'
Coal Production - mt
Coal Production - EJ
Coal Consumption - EJ
Coal & Uranium - Prices
error: '2024'
Coal - Trade movements
error: "Cannot get right slice bound for non-unique label: 'total_world'"
Coal trade movements 2024
error: '2024'
Nuclear Generation - TWh
Nuclear Consumption - EJ
Hydro Gen

Could not determine dtype for column 7, falling back to string


Renewables Consumption -EJ
Renewable Power (inc hydro) -EJ
Ren power (excl hydro) - EJ
Renewable Power (inc hydro)-TWh
Ren power (excl hydro) - TWh
Renewables Generation by Source
error: '2024'
Solar Generation - TWh
Solar Consumption - EJ
Solar Installed Capacity
error: '2024'
Wind Generation - TWh
Wind Consumption - EJ
Wind Installed Capacity
error: '2024'
Wind and Solar Technology Types
error: '2024'
Geo Biomass Other - TWh
Geo Biomass Other - EJ
Biofuels production - kboed
error: "Cannot get right slice bound for non-unique label: 'total_world'"
Biofuels production - PJ
error: "Cannot get right slice bound for non-unique label: 'total_world'"
Biofuels Consumption - kboed
error: "Cannot get right slice bound for non-unique label: 'total_world'"
Biofuels consumption - PJ
error: "Cannot get right slice bound for non-unique label: 'total_world'"
Electricity Generation - TWh
Elec generation by fuel
error: '2024'
Oil inputs - Elec generation 
Gas inputs - Elec generation
Coal inputs - El

Could not determine dtype for column 34, falling back to string
Could not determine dtype for column 34, falling back to string
Could not determine dtype for column 34, falling back to string
Could not determine dtype for column 34, falling back to string
Could not determine dtype for column 1, falling back to string


Cobalt P-R
Lithium P-R
Natural Graphite P-R
Rare Earth metals P-R
Copper P-R
Manganese P-R
Nickel P-R
Zinc P-R
Platinum Group Metals P-R
Bauxite P-R
Aluminium P-R
Tin P-R
Vanadium P-R
Mineral Commodity Prices
error: '2024'
Primary Energy Cons (old meth)
PE Cons by fuel (old meth) 
error: '2024'


In [228]:
data1 = preprocess(data1)

In [229]:
data1.head()

,geo,geo_name,1965,1966,1967,1968,1969,1970,1971,1972,...,2015,2016,2017,2018,2019,2019.1,2008-18,2019.2,Unnamed: 59,Unnamed: 60
0,canada,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,...,13.993650,13.936678,14.109482,14.349534,14.214058,-0.009441,0.006404,0.024343,NaN,NaN
1,mexico,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,...,7.688300,7.785419,7.895341,7.833322,7.720742,-0.014372,0.009057,0.013223,NaN,NaN
2,us,US,52.433795,55.383038,57.311776,60.799094,64.049612,66.215508,67.591944,71.064330,...,92.149004,92.017799,92.330525,95.602919,94.648804,-0.009980,0.001054,0.162098,NaN,NaN
3,total_north_america,Total North America,58.408365,61.733704,63.924460,67.868147,71.589254,74.284923,75.943920,80.093438,...,113.830955,113.739897,114.335348,117.785775,116.583604,-0.010206,0.002204,0.199664,NaN,NaN
4,argentina,Argentina,1.127632,1.165108,1.199725,1.241234,1.283912,1.212762,1.288378,1.311442,...,3.585803,3.576195,3.566920,3.540866,3.462769,-0.022056,0.012502,0.005930,NaN,NaN


In [230]:
process_1(data1, to_concept_id('Primary Energy: Consumption - Mtoe')).head(100)

,geo,year,primary_energy_consumption_mtoe
3520,algeria,1965,0.08913947
3521,algeria,1966,0.10716806
3522,algeria,1967,0.10142615
3523,algeria,1968,0.10887872
3524,algeria,1969,0.12050818
...,...,...,...
260,argentina,2005,2.802798
261,argentina,2006,2.94482844
262,argentina,2007,3.07479095
263,argentina,2008,3.12715637


In [231]:
df = process_1(data1, to_concept_id('Primary Energy: Consumption - Mtoe'))

In [232]:
data1['geo_name'].unique()

array(['Canada', 'Mexico', 'US', 'Total North America', 'Argentina',
       'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Peru',
       'Trinidad & Tobago', 'Venezuela', 'Central America',
       'Other Caribbean', 'Other South America',
       'Total S. & Cent. America', 'Austria', 'Belgium', 'Bulgaria',
       'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia',
       'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland',
       'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
       'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal',
       'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
       'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom',
       'Other Europe', 'Total Europe', 'Azerbaijan', 'Belarus',
       'Kazakhstan', 'Russian Federation', 'Turkmenistan', 'USSR',
       'Uzbekistan', 'Other CIS', 'Total CIS', 'Iran', 'Iraq', 'Israel',
       'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates',
       'Other Middle

In [233]:
# df.to_csv('../../ddf--datapoints--primary_energy_consumption_mtoe--by--geo--year.csv', index=False)

In [234]:
countries = []

In [235]:
tabs_indicator_mapping1 = {
    'Primary Energy Consumption': to_concept_id('Primary Energy Consumption'),
    'Primary Energy - Cons capita': to_concept_id('Primary Energy Consumption per capita'),
    'Carbon Dioxide Emissions': to_concept_id('Carbon Dioxide Emissions'),
    'Oil - Refinery throughput': to_concept_id('Oil - Refinery throughput'),
    'Oil - Refining capacity': to_concept_id('Oil - Refining capacity'),
    'Oil - Proved reserves history': to_concept_id('Oil - Proved reserves'),
    'Gas - Proved reserves history ': to_concept_id('Gas - Proved reserves'),
    'Electricity Generation ': to_concept_id('Electricity Generation')
}

In [236]:
for t, indicator in tabs_indicator_mapping1.items():
    data = sheet_loader(source_file, sheet_name=t)
    data = preprocess(data)
    countries.append(data['geo_name'].unique())
    df = process_1(data, indicator)
    df.to_csv('../../ddf--datapoints--{}--by--geo--year.csv'.format(indicator), index=False)

In [237]:
def make_dict1(sheet, unit, fuel):
    return dict(sheet=sheet, unit=unit, fuel=fuel)

In [238]:
#Mtoe unit is no longer used. Its in Exajoules

tabs_indicator_mapping2 = {
    'production': [
        make_dict1(sheet='Gas Production - Bcm', unit='bcm', fuel='gas'),
        make_dict1(sheet='Gas Production - Bcf', unit='bcf', fuel='gas'),
        # make_dict1('Gas Production - Mtoe', 'mtoe', 'gas'),
        make_dict1('Gas Production - EJ', 'exajoules', 'gas'),
        make_dict1('Coal Production - Tonnes', 'tonne', 'coal'),
        # make_dict1('Coal Production - Mtoe', 'mtoe', 'coal'),
        make_dict1('Coal Production - EJ', 'exajoules', 'coal'),
        
        #The Bio Fuels sheet has multiple tables in their sheet like BioDiesel, BioGasoline so they we created
        #another ETL biofuels_run.ipynb for it.
        
        #make_dict1('Biofuels Production - Kboed', 'kboed', 'biofuel'),
        #make_dict1('Biofuels Production - Ktoe', 'ktoe', 'biofuel'),
        make_dict1('Oil Production - Barrels', 'barrel', 'oil'),
        make_dict1('Oil Production - Tonnes', 'tonne', 'oil')
    ],
    'consumption': [
        make_dict1('Gas Consumption - Bcm', 'bcm', 'gas'),
        make_dict1('Gas Consumption - Bcf', 'bcf', 'gas'),
        #make_dict1('Gas Consumption - Mtoe', 'mtoe', 'gas'),
        make_dict1('Gas Consumption - EJ', 'exajoules', 'gas'),
        #make_dict1('Coal Consumption - Mtoe', 'mtoe', 'coal'),
        make_dict1('Coal Consumption - EJ', 'exajoules', 'coal'),
        #make_dict1('Nuclear Consumption - Mtoe', 'mtoe', 'nuclear'),
        make_dict1('Nuclear Consumption - EJ', 'exajoules', 'nuclear'),
        #make_dict1('Hydro Consumption - Mtoe', 'mtoe', 'hydro'),
        make_dict1('Hydro Consumption - EJ', 'exajoules', 'hydro'),
        # make_dict1('Other renewables - Mtoe', 'mtoe', 'other_renewables'),
        #make_dict1('Solar Consumption - Mtoe', 'mtoe', 'solar'),
        make_dict1('Solar Consumption - EJ', 'exajoules', 'solar'),
        #make_dict1('Wind Consumption - Mtoe', 'mtoe', 'wind'),
        make_dict1('Wind Consumption - EJ', 'exajoules', 'wind'),
        #make_dict1('Geo Biomass Other - Mtoe', 'mtoe', 'geo_biomass_other'),
        make_dict1('Geo Biomass Other - EJ', 'exajoules', 'geo_biomass_other'),
        make_dict1('Oil Consumption - Barrels', 'barrel', 'oil'),
        make_dict1('Oil Consumption - Tonnes', 'tonne', 'oil'),
        #make_dict1('Oil Consumption - Mtoe', 'mtoe', 'oil')
        make_dict1('Oil Consumption - EJ', 'exajoules', 'oil')
    ],
    'electricity_generation':[
        make_dict1('Elec Gen from Oil', 'twh', 'oil'),
        make_dict1('Elec Gen from Gas', 'twh', 'gas'),
        make_dict1('Elec Gen from Coal', 'twh', 'coal'),
        make_dict1('Elec Gen from Other', 'twh', 'other'),
        make_dict1('Nuclear Generation - TWh', 'twh', 'nuclear'),
        make_dict1('Hydro Generation - TWh', 'twh', 'hydro'),
        # make_dict1('Other renewables - TWh', 'twh', 'other_renewables'),
        make_dict1('Solar Generation - TWh', 'twh', 'solar'),
        make_dict1('Wind Generation -TWh', 'twh', 'wind'),
        make_dict1('Geo Biomass Other - TWh', 'twh', 'geo_biomass_other')
    ]
}

In [239]:
for i, ms in tabs_indicator_mapping2.items():
    #Hisham: I made this datapoint indicator--by--geo--year.csv
    #Previously it was datapoints--indicator--by--geo--fuel--year--unit.csv
    for m in ms:
        data = []
        d = sheet_loader(source_file, sheet_name=m['sheet'])
        d = preprocess(d)        
        countries.append(d['geo_name'].unique())
        indicator_name = m['fuel'] + '_' + i + '_' + m['unit']
        d = process_1(d, indicator_name)        
        #d['fuel'] = m['fuel']
        #d['unit'] = m['unit']
        data.append(d)
        data = pd.concat(data, ignore_index=True)
        df = data[['geo', 'year', indicator_name]]    
        df[indicator_name] = df[indicator_name].map(number_formatter)
        df = df.sort_values(by=['geo', 'year'])
        
        df.to_csv('../../ddf--datapoints--{}--by--geo--year.csv'.format(m['fuel'] + '_' + i + '_' + m['unit']), index=False)
        
    '''data = pd.concat(data, ignore_index=True)
    df = data[['geo', 'fuel', 'year', 'unit', i]]    
    df[i] = df[i].map(number_formatter)
    df = df.sort_values(by=['fuel', 'unit', 'geo', 'year'])
    print(df.head(100))
    df.to_csv('../../ddf--datapoints--{}--by--geo--fuel--year--unit.csv'.format(i), index=False)'''

In [240]:
#biofuels_run.ipynb is pasted here in this cell, to make this the complete ETL.

countries4 = []

def preprocess4(data,i):
    """preprocessing the data:
    1. rename the first column to geo_name
    2. rename the geo_name to alphanumeric
    3. drop all empty lines and lines after 'total world'

    Note: This function only applies to the tab with country as row index
    and year as column index.
    """
    
    data = data.rename(columns={data.columns[0]: 'geo_name'})
    data['geo'] = data['geo_name'].map(to_concept_id)
    #data = data.dropna(how='all')
    slice_array = data.query('geo == "total_world"').index
    slice_fuel_array = data.query('geo == @i').index
    
    
    #data = data.loc[:'total_world']
    fuel_index = biofuels_index[i]
    
    if (slice_fuel_array.empty):
        data = data.iloc[:slice_array[fuel_index]+1]
    else:
        data = data.iloc[slice_fuel_array[0]+1:slice_array[fuel_index]+1]
    data = data.dropna(how='all')
    data = data.set_index('geo')
    data = data.reset_index()
    return data


def make_dict4(sheet, unit, fuel):
    return dict(sheet=sheet, unit=unit, fuel=fuel)

def process_4(data, ddf_id):
    data = data.dropna(axis=1, how='all')
    data = data.drop('geo_name', axis=1)
    idx = list(data.columns).index(2019)  #TODO: change the year column
    data = data.iloc[:, :idx + 1]  # drop columns after latest year of each sheet.
    #data = data.drop(['2019.1', '2019.2', '2008-18'], axis=1)
    
    data = data.set_index('geo')

    d = data.T.unstack()
    d = d.dropna()
    d = d.reset_index()
    d.columns = ['geo', 'year', ddf_id]
    d[ddf_id] = d[ddf_id].map(number_formatter)

    return d.sort_values(by=['geo', 'year'])

tabs_indicator_mapping = {'production':[
    make_dict4(sheet='Biofuels Production - Kboed', unit='kboed', fuel='biofuel'),
    make_dict4(sheet='Biofuels Production - PJ', unit='petajoules', fuel='biofuel'),
    make_dict4(sheet='Biofuels Production - Kboed', unit='kboed', fuel='biogasoline'),
    make_dict4(sheet='Biofuels Production - PJ', unit='petajoules', fuel='biogasoline'),
    make_dict4(sheet='Biofuels Production - Kboed', unit='kboed', fuel='biodiesel'),
    make_dict4(sheet='Biofuels Production - PJ', unit='petajoules', fuel='biodiesel'),
    
],
                         'consumption':[
    make_dict4(sheet='Biofuels Consumption - Kboed', unit='kboed', fuel='biofuel'),
    make_dict4(sheet='Biofuels Consumption - PJ', unit='petajoules', fuel='biofuel'),
    make_dict4(sheet='Biofuels Consumption - Kboed', unit='kboed', fuel='biogasoline'),
    make_dict4(sheet='Biofuels Consumption - PJ', unit='petajoules', fuel='biogasoline'),
    make_dict4(sheet='Biofuels Consumption - Kboed', unit='kboed', fuel='biodiesel'),
    make_dict4(sheet='Biofuels Consumption - PJ', unit='petajoules', fuel='biodiesel'),
                             
],                             
                             }

biofuels_index = {'biofuel':0,'biogasoline':1,'biodiesel':2}

for i, ms in tabs_indicator_mapping.items():
    
    for m in ms:
        data = []
        d = sheet_loader(source_file, sheet_name=m['sheet'])
        d = preprocess4(d,m['fuel'])
        #print(d.query('geo == "total_world"').index[0])
        #print(biofuels_index[i])
        countries4.append(d['geo_name'].unique())
        indicator_name = m['fuel'] + '_' + i + '_' + m['unit']
        d = process_4(d, indicator_name)
        #d['fuel'] = m['fuel']
        #d['unit'] = m['unit']
        data.append(d)
        data = pd.concat(data, ignore_index=True)
        df = data[['geo', 'year', indicator_name]]
        df[indicator_name] = df[indicator_name].map(number_formatter)
        df = df.sort_values(by=['geo', 'year'])
        df.to_csv('../../ddf--datapoints--{}--by--geo--year.csv'.format(m['fuel'] + '_' + i + '_' + m['unit']), index=False)
        
        
        

        

In [241]:
c4 = np.concatenate(countries4)

c4 = pd.DataFrame({'name': c4})

c4['name'] = c4['name'].str.strip()

c4 = c4.drop_duplicates(subset='name')
c4['geo'] = c4['name'].map(to_concept_id)



In [242]:
c4[~c4.geo.isin(c1.geo)]

,name,geo
70,Canada & Mexico,canada_mexico
74,Europe,europe
75,CIS,cis
76,Middle East,middle_east
77,Africa,africa
78,Asia Pacific,asia_pacific


In [243]:
import numpy as np

In [244]:
countries = np.concatenate(countries)

In [245]:
countries

array(['Canada', 'Mexico', 'US', ..., 'Other Asia Pacific',
       'Total Asia Pacific', 'Total World'], dtype=object)

In [246]:
c1 = pd.DataFrame({'name': countries})

In [247]:
c1['name'] = c1['name'].str.strip()

In [248]:
c1 = c1.drop_duplicates(subset='name')

In [249]:
c1['geo'] = c1['name'].map(to_concept_id)

In [250]:
c1 = c1[['geo', 'name']].sort_values(by='geo')

In [251]:
c1.head()

,geo,name
70,algeria,Algeria
516,angola,Angola
4,argentina,Argentina
80,australia,Australia
16,austria,Austria


In [252]:
c1[c1['name'] == 'North Macedonia']

,geo,name
36,north_macedonia,North Macedonia


In [253]:
!open ../../

/bin/bash: open: command not found


In [254]:
# Reserves by key materials

In [255]:
def preprocess_2(data):
    """preprocessing the data:
    1. rename the first column to geo_name
    2. rename the geo_name to alphanumeric
    3. drop all empty lines and lines after 'total world'

    Note: This function only applies to the tab with country as row index
    and year as column index.
    """
    data = data.rename(columns={data.columns[0]: 'geo_name'})
    data['geo'] = data['geo_name'].map(to_concept_id)
    data = data.set_index('geo')
    data = data.dropna(how='all')
    if 'world' in data.index:
        data = data.loc[:'world']
    else:
        data = data.loc[:'total_world']
    data = data.reset_index()
    return data

def process_2(data, ddf_id):
    data = data.dropna(axis=1, how='all')
    data = data.drop('geo_name', axis=1)
    idx = list(data.columns).index(2019)  # TODO
    data = data.iloc[:, :idx + 1]  # drop columns after latest year of each sheet.
    #data = data.drop(['2017.1', '2017.2', '2006-16'], axis=1)
    
    data = data.set_index('geo')

    d = data.T.unstack()
    d = d.dropna()
    d = d.reset_index()
    d.columns = ['geo', 'year', ddf_id]

    return d.sort_values(by=['geo', 'year'])

In [256]:
countries2 = []

In [257]:
def create_datapoint_2(data_tabs, fuels, indicator):
    data = []

    for t, f in zip(data_tabs, fuels):
        d = sheet_loader(source_file, sheet_name=t)
        d = preprocess_2(d)
        countries2.append(d['geo_name'].unique())
        d = process_2(d, indicator)
        d['material'] = f
        data.append(d)
    data = pd.concat(data, ignore_index=True)
    df = data[['geo', 'material', 'year', indicator]]
    df[indicator] = df[indicator].map(number_formatter)
    # df.to_csv('../../ddf--datapoints--{}--by--geo--mineral--year.csv'.format(indicator), index=False)
    
    return df

In [258]:
data_tabs = ['Cobalt Production-Reserves', 
             'Lithium Production-Reserves',
             'Graphite Production-Reserves',
             'Rare Earth Production-Reserves'
            ]
fuels  = ['cobalt', 'lithium', 'graphite', 'rare_earth']
indicator = 'production_reserve'

In [259]:
df = create_datapoint_2(data_tabs, fuels, indicator)

In [260]:
countries2 = np.concatenate(countries2)

In [261]:
c2 = pd.DataFrame({'name': countries2})

In [262]:
c2['name'] = c2['name'].str.strip()

In [263]:
c2 = c2.drop_duplicates(subset='name')

In [264]:
c2['geo'] = c2['name'].map(to_concept_id)

In [265]:
c2[~c2.geo.isin(c1.geo)]

,name,geo
2,DR Congo,dr_congo
3,Cuba,cuba
4,Madagascar,madagascar
6,New Caledonia,new_caledonia
11,Zambia,zambia
12,Rest of World,rest_of_world
24,Brazil1,brazil1
27,India2,india2
30,Mozambique,mozambique
35,Rest of World3,rest_of_world3


In [266]:
df.loc[df.geo == 'brazil1', 'geo'] = 'brazil'
df.loc[df.geo == 'india2', 'geo'] = 'india'
df.loc[df.geo == 'rest_of_world2', 'geo'] = 'rest_of_world'
df.loc[df.geo == 'rest_of_world3', 'geo'] = 'rest_of_world'

In [267]:
df.loc[df.geo == 'world', 'geo'] = 'total_world'

In [268]:
# df = df[~df.geo.isin(['rest_of_world'])]

In [269]:
df.sample(10)

,geo,material,year,production_reserve
992,rest_of_world,rare_earth,2017,0.25
1058,total_world,rare_earth,2019,209.599
172,papua_new_guinea,cobalt,2017,3.31
366,australia,lithium,2019,40.71428571
540,zimbabwe,lithium,1995,0.52
557,zimbabwe,lithium,2012,1.06
306,zambia,cobalt,2009,5.879
1016,russian_federation,rare_earth,2016,3.1
872,zimbabwe,graphite,2019,2
1021,thailand,rare_earth,1996,0


In [270]:
df.geo.unique() 

array(['australia', 'canada', 'cuba', 'dr_congo', 'madagascar', 'morocco',
       'new_caledonia', 'papua_new_guinea', 'philippines',
       'rest_of_world', 'russian_federation', 'south_africa',
       'total_world', 'zambia', 'argentina', 'brazil', 'chile', 'china',
       'portugal', 'us', 'zimbabwe', 'india', 'mexico', 'mozambique',
       'sri_lanka', 'ukraine', 'malaysia', 'thailand'], dtype=object)

In [271]:
(df.sort_values(by=['material', 'geo', 'year'])
 .to_csv('../../ddf--datapoints--production_reserve--by--geo--material--year.csv', index=False))

In [272]:
sheet_loader2 = partial(pd.read_excel, skiprows=3)

In [273]:
countries3 = []

In [274]:
def create_datapoint_3(data_tabs, fuels, indicator):
    data = []

    for t, f in zip(data_tabs, fuels):
        d = sheet_loader2(source_file, sheet_name=t)
        d = preprocess_2(d)
        countries3.append(d['geo_name'].unique())
        d = process_2(d, indicator)
        d['renewable'] = f
        data.append(d)
    data = pd.concat(data, ignore_index=True)
    df = data[['geo', 'renewable', 'year', indicator]]
    df[indicator] = df[indicator].map(number_formatter)
    # df.to_csv('../../ddf--datapoints--{}--by--geo--mineral--year.csv'.format(indicator), index=False)
    
    return df

In [275]:
data_tabs = ['Geothermal Capacity', 
             'Solar Capacity',
             'Wind Capacity'
            ]
fuels  = ['geothermal', 'solar', 'wind']
indicator = 'installed_capacity'

In [276]:
df = create_datapoint_3(data_tabs, fuels, indicator)

In [277]:
df.sample(10)

,geo,renewable,year,installed_capacity
974,czech_republic,solar,2007,4
2388,denmark,wind,2016,5245.563
3367,tunisia,wind,2016,245
1807,thailand,solar,2000,0
1156,india,solar,1997,0
3378,turkey,wind,2004,19
2234,brazil,wind,2000,22
2928,philippines,wind,2010,33
2438,france,wind,1997,13
2297,canada,wind,2017,12403


In [278]:
c3 = np.concatenate(countries3)

In [279]:
c3 = pd.DataFrame({'name': c3})

In [280]:
c3['name'] = c3['name'].str.strip()

In [281]:
c3 = c3.drop_duplicates(subset='name')
c3['geo'] = c3['name'].map(to_concept_id)

In [282]:
print(c3[~c3.geo.isin(c1.geo)][['geo', 'name']].to_csv(index=False))

geo,name
costa_rica,Costa Rica
el_salvador,El Salvador
guatemala,Guatemala
honduras,Honduras
nicaragua,Nicaragua
other_s_and_cent_america,Other S. and Cent. America
ethiopia,Ethiopia
kenya,Kenya
jordan,Jordan
uruguay,Uruguay



In [283]:
df.loc[df.geo == 'russian_fed', 'geo'] = 'russian_federation'

In [284]:
(df.sort_values(by=['renewable', 'geo', 'year'])
 .to_csv('../../ddf--datapoints--{}--by--geo--renewable--year.csv'.format(indicator), index=False))

In [285]:
m2 = {'brazil1': 'brazil', 'india2': 'india', 
      'rest_of_world2': 'rest_of_world', 
      'rest_of_world3': 'rest_of_world'}
c2['geo'] = c2['geo'].map(partial(apply_map, m=m2))

In [286]:
m3 = {'russian_fed': 'russian_federation'}
c3['geo'] = c3['geo'].map(partial(apply_map, m=m3))

In [287]:
#added c4 from biofuels_run.ipynb
geo_ent = pd.concat([c1, c2, c3, c4], ignore_index=True, sort=True)

In [288]:
geo_ent[geo_ent['name'] == 'North Macedonia']

,geo,name
62,north_macedonia,North Macedonia


In [289]:
(geo_ent
 .drop_duplicates(subset='geo')
 .sort_values(by='name')
 .to_csv('../../ddf--entities--geo.csv', index=False))

In [290]:
units = ['Bcm', 'Bcf', 'Exajoules', 'Barrel', 'Tonne', 'TWh', 'Kboed', 'Petajoules']

In [291]:
units_df = pd.DataFrame({'unit': list(map(to_concept_id, units)), 'name': units})

In [292]:
units_df

,unit,name
0,bcm,Bcm
1,bcf,Bcf
2,exajoules,Exajoules
3,barrel,Barrel
4,tonne,Tonne
5,twh,TWh
6,kboed,Kboed
7,petajoules,Petajoules


In [293]:
units_df.to_csv('../../ddf--entities--unit.csv', index=False)

In [294]:
# 

In [295]:
!cp fuel.csv ../../ddf--entities--fuel.csv

In [296]:
materials = pd.DataFrame({'material': ['cobalt', 'lithium', 'graphite', 'rare_earth'], 
                     'name': ['Cobalt', 'Lithium', 'Graphite', 'Rare Earth']})

In [297]:
materials.to_csv('../../ddf--entities--material.csv', index=False)

In [298]:
conc1 = []
conc1_name = []

for n, i in tabs_indicator_mapping1.items():
    conc1.append(i)
    conc1_name.append(n)

In [299]:
concs2 = ['fuel_production', 'fuel_consumption', 'electricity_generation', 'production_reserve', 'installed_capacity']
concs2_name = ['Fuel Production', 'Fuel Consumption', 'Electricity Generation', 'Production Reserve', 'Installed Capacity']

In [300]:
concs = [*conc1, *concs2]
concs_name = [*conc1_name, *concs2_name]

In [301]:
concs

['primary_energy_consumption',
 'primary_energy_consumption_per_capita',
 'carbon_dioxide_emissions',
 'oil_refinery_throughput',
 'oil_refining_capacity',
 'oil_proved_reserves',
 'gas_proved_reserves',
 'electricity_generation',
 'fuel_production',
 'fuel_consumption',
 'electricity_generation',
 'production_reserve',
 'installed_capacity']

In [302]:
measures = pd.DataFrame({'concept': concs, 'name': concs_name})

In [303]:
measures['concept_type'] = 'measure'

In [304]:
measures = measures.drop_duplicates(subset='concept')

In [305]:
measures.to_csv('../../ddf--concepts--continuous.csv', index=False)

In [306]:
disc = pd.DataFrame([
    ['name', 'Name', 'string', ''],
    ['year', 'Year', 'time', ''],
    ['geo', 'Geo', 'entity_domain', ''],
    ['unit', 'Unit', 'entity_domain', ''],
    ['fuel', 'Fuel', 'entity_domain', ''],
    ['renewable', 'Renewables', 'entity_set', 'fuel'],
    ['material', 'Material', 'entity_domain', ''],
    ['domain', 'Domain', 'string', '']
], columns=['concept', 'name', 'concept_type', 'domain'])

In [307]:
disc

,concept,name,concept_type,domain
0,name,Name,string,
1,year,Year,time,
2,geo,Geo,entity_domain,
3,unit,Unit,entity_domain,
4,fuel,Fuel,entity_domain,
5,renewable,Renewables,entity_set,fuel
6,material,Material,entity_domain,
7,domain,Domain,string,


In [308]:
disc.to_csv('../../ddf--concepts--discrete.csv', index=False)